## Wrangle and Analyze Data

Real-world data rarely comes clean. Using Python and its libraries, we will gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it.


## Table of Contents
- [Introduction](#intro)
- [Part I - Gathering Data](#gather)
- [Part II - Assessing Data](#assess)
- [Part III - Cleaning Data](#clean)


<a id='intro'></a>
### Introduction

For this project I will be wrangling WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive contains only very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations. 

WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

<a id='gather'></a>
#### Part I - Gathering Data

In this section I will gather data using 3 methodologies:
1. *The WeRateDogs Twitter archive*. __Downloaded manually__ from the server. 

2. *The tweet image predictions*, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be __downloaded programmatically__ using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3. Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, __query the Twitter API__ for each tweet's JSON data __using Python's Tweepy library__ and store each tweet's entire set of JSON data in a file called *tweet_json.txt* file.


Let's start with importing neccessary libraries.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import requests
import os
import tweepy
import json

In [2]:
# Read WeRateDogs Twitter archive csv file
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# First check if the data is properly loaded
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Download tweet image predictions tsv file from the Udacity's server

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

with open(url.split('/')[-1], 'wb') as f:
    f.write(r.content)

In [4]:
# Read the tweet image predictions tsv file
twitter_images = pd.read_csv('image-predictions.tsv', sep = '\t')

In [5]:
# First check if the data is properly loaded
twitter_images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### Twitter authentification setup

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

#### Download Tweets part 1

--- List for storing json outputs
tweet_list = []
--- List for storing Twitter IDs where the status has not been found
tweet_errors = []

--- Loop through Twitter IDs from the twitter archive file
--- Append json outputs to tweet_list and not found Twitter IDs to 'tweet_errors' list
for tweet_id in twitter_archive.tweet_id:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        tweet_list.append(tweet._json)
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        tweet_errors.append(tweet_id)

#### Download Tweets part 2

--- Additional list for storing Twitter IDs where the status has not been found the second time
missing_list = []

--- Loop through the list of missing IDs
--- Append json outputs to tweet_list and not found Twitter IDs to missing_list
for missing_id in tweet_errors:
    try:
        tweet_list.append(tweet._json)
    except Exception as ex:
        print(str(missing_id)+ " _ " + str(ex))
        missing_list.append(ex)

#### Create and safe dataframe
--- Create DataFrames from list of dictionaries
json_df = pd.DataFrame(tweet_list)

--- Save the dataFrame in file
json_df.to_csv('tweet_json.txt', index=False)

In [6]:
# Read tweet_json.txt file
twitter_json = pd.read_csv('tweet_json.txt', encoding = 'utf-8')

In [7]:
# First check if the data is properly loaded
twitter_json.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,Tue Aug 01 16:23:56 +0000 2017,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",38126,False,This is Phineas. He's a mystical boy. Only eve...,NaN,...,NaN,NaN,NaN,NaN,8339,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1,NaN,NaN,Tue Aug 01 00:17:27 +0000 2017,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...",32703,False,This is Tilly. She's just checking pup on you....,NaN,...,NaN,NaN,NaN,NaN,6162,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2,NaN,NaN,Mon Jul 31 00:18:03 +0000 2017,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...",24623,False,This is Archie. He is a rare Norwegian Pouncin...,NaN,...,NaN,NaN,NaN,NaN,4078,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
3,NaN,NaN,Sun Jul 30 15:58:51 +0000 2017,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...",41473,False,This is Darla. She commenced a snooze mid meal...,NaN,...,NaN,NaN,NaN,NaN,8484,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
4,NaN,NaN,Sat Jul 29 16:00:24 +0000 2017,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...",39639,False,This is Franklin. He would like you to stop ca...,NaN,...,NaN,NaN,NaN,NaN,9168,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


<a id='assess'></a>
### Part II - Assessing  Data

After gathering each of the above pieces of data, our task is to assess them visually and programmatically for quality and tidiness issues. We should detect and document at least eight (8) quality issues and two (2) tidiness issues in the wrangle_act.ipynb Jupyter Notebook.

#### Step 1: Detect

**a. Visual Assesment**

In [8]:
# Visual assesment of the 1st dataframe: twitter-archive-enhanced.csv
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [9]:
# Visual assesment of the 2nd dataframe: image-predictions.tsv
twitter_images

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [10]:
# Visual assesment of the 3rd dataframe: tweet_json.txt
twitter_json

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,Tue Aug 01 16:23:56 +0000 2017,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",38126,False,This is Phineas. He's a mystical boy. Only eve...,NaN,...,NaN,NaN,NaN,NaN,8339,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1,NaN,NaN,Tue Aug 01 00:17:27 +0000 2017,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...",32703,False,This is Tilly. She's just checking pup on you....,NaN,...,NaN,NaN,NaN,NaN,6162,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2,NaN,NaN,Mon Jul 31 00:18:03 +0000 2017,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...",24623,False,This is Archie. He is a rare Norwegian Pouncin...,NaN,...,NaN,NaN,NaN,NaN,4078,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
3,NaN,NaN,Sun Jul 30 15:58:51 +0000 2017,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...",41473,False,This is Darla. She commenced a snooze mid meal...,NaN,...,NaN,NaN,NaN,NaN,8484,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
4,NaN,NaN,Sat Jul 29 16:00:24 +0000 2017,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...",39639,False,This is Franklin. He would like you to stop ca...,NaN,...,NaN,NaN,NaN,NaN,9168,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
5,NaN,NaN,Sat Jul 29 00:08:17 +0000 2017,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891087942176911360, 'id_str'...",19902,False,Here we have a majestic great white breaching ...,NaN,...,NaN,NaN,NaN,NaN,3055,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
6,NaN,NaN,Fri Jul 28 16:27:12 +0000 2017,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890971906207338496, 'id_str'...",11637,False,Meet Jax. He enjoys ice cream so much he gets ...,NaN,...,NaN,NaN,NaN,NaN,2026,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
7,NaN,NaN,Fri Jul 28 00:22:40 +0000 2017,"[0, 118]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890729118844600320, 'id_str'...",64324,False,When you watch your owner call another dog a g...,NaN,...,NaN,NaN,NaN,NaN,18508,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
8,NaN,NaN,Thu Jul 27 16:25:51 +0000 2017,"[0, 122]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 890609177319665665, 'id_str'...",27352,False,This is Zoey. She doesn't want to be one of th...,NaN,...,NaN,NaN,NaN,NaN,4194,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
9,NaN,NaN,Wed Jul 26 15:59:51 +0000 2017,"[0, 133]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 890240245463175168, 'id_str'...",31382,False,This is Cassie. She is a college pup. Studying...,NaN,...,NaN,NaN,NaN,NaN,7240,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


**a. Programmatic Assessment**

In [11]:
# View 10 random records from the twitter_archive dataframe
twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2154,669625907762618368,NaN,NaN,2015-11-25 21:17:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Penny. She's having fun AND being safe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669625907...,12,10,Penny,None,None,None,None
809,771500966810099713,NaN,NaN,2016-09-02 00:12:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dakota. He's just saying hi. That's al...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/771500966...,12,10,Dakota,None,None,None,None
601,798665375516884993,NaN,NaN,2016-11-15 23:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lola. She fell asleep o...,6.718968e+17,4.196984e+09,2015-12-02 03:40:57 +0000,https://twitter.com/dog_rates/status/671896809...,10,10,Lola,None,None,None,None
1335,705239209544720384,NaN,NaN,2016-03-03 03:51:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jimothy. He lost his body during the t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/705239209...,11,10,Jimothy,None,None,None,None
480,815736392542261248,NaN,NaN,2017-01-02 01:48:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Akumi. It's his birthday. He received ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/815736392...,11,10,Akumi,None,None,None,None
255,844223788422217728,NaN,NaN,2017-03-21 16:26:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Margo. She just dug pup a massive hole...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/844223788...,12,10,Margo,None,None,None,None
350,831552930092285952,NaN,NaN,2017-02-14 17:17:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dutch. He dressed up as his favorite e...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/831552930...,13,10,Dutch,None,None,None,None
1217,714962719905021952,NaN,NaN,2016-03-29 23:49:30 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Just in case anyone's having a bad day. 12/10 ...,NaN,NaN,NaN,https://vine.co/v/inVtemLt9tE,12,10,None,None,None,None,None
2161,669564461267722241,NaN,NaN,2015-11-25 17:13:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Coriander Baton Rouge named Alfredo....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669564461...,10,10,a,None,None,None,None
278,840268004936019968,NaN,NaN,2017-03-10 18:27:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Monty. He makes instantly regrettable ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/840268004...,12,10,Monty,None,None,None,None


In [12]:
# View info of twitter_archive dataframe
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
# View descriptive statistics of twitter_archive dataframe
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [14]:
# View 10 random records from the twitter_images dataframe
twitter_images.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
138,668537837512433665,https://pbs.twimg.com/media/CUcfnWlWsAAzlwE.jpg,1,Lakeland_terrier,0.372988,True,toy_poodle,0.250445,True,Chihuahua,0.189737,True
756,688519176466644993,https://pbs.twimg.com/media/CY4ciRFUMAAovos.jpg,1,Pembroke,0.696372,True,Cardigan,0.121052,True,Shetland_sheepdog,0.050592,True
388,673355879178194945,https://pbs.twimg.com/media/CVg9mTYWIAAu7J6.jpg,1,Rottweiler,0.529248,True,miniature_pinscher,0.168296,True,Appenzeller,0.100452,True
2050,887343217045368832,https://pbs.twimg.com/ext_tw_video_thumb/88734...,1,Mexican_hairless,0.330741,True,sea_lion,0.275645,False,Weimaraner,0.134203,True
1523,788150585577050112,https://pbs.twimg.com/media/CvASw6dWcAQmo3X.jpg,3,chow,0.814145,True,Pomeranian,0.112704,True,Chihuahua,0.015883,True
553,677573743309385728,https://pbs.twimg.com/media/CWc5uVPXIAErLYr.jpg,2,patio,0.535070,False,folding_chair,0.080419,False,parallel_bars,0.034796,False
2068,890971913173991426,https://pbs.twimg.com/media/DF1eOmZXUAALUcq.jpg,1,Appenzeller,0.341703,True,Border_collie,0.199287,True,ice_lolly,0.193548,False
827,693486665285931008,https://pbs.twimg.com/ext_tw_video_thumb/69348...,1,sea_lion,0.519811,False,Siamese_cat,0.290971,False,black-footed_ferret,0.039967,False
1729,820749716845686786,https://pbs.twimg.com/media/C2PjgjQXcAAc4Uu.jpg,2,golden_retriever,0.838012,True,Pekinese,0.056733,True,Labrador_retriever,0.023944,True
1726,820314633777061888,https://pbs.twimg.com/media/C2JXyARUAAE4gbL.jpg,2,Gordon_setter,0.940724,True,black-and-tan_coonhound,0.042041,True,Rottweiler,0.009417,True


In [15]:
# View info of twitter_images dataframe
twitter_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [16]:
# View descriptive statistics of twitter_json dataframe
twitter_images.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [17]:
# View 10 random records from the twitter_json dataframe
twitter_json.sample(10)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
2200,NaN,NaN,Sun Nov 22 21:41:02 +0000 2015,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 668544740393136128, 'id_str'...",532,False,It is an honor to rate this pup. He is a Snork...,NaN,...,NaN,NaN,NaN,NaN,233,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1680,NaN,NaN,Mon Dec 28 01:12:59 +0000 2015,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 681281653780656128, 'id_str'...",3389,False,Meet Sarge. His parents signed him up for danc...,NaN,...,NaN,NaN,NaN,NaN,1180,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1163,NaN,NaN,Tue Apr 12 01:51:36 +0000 2016,"[0, 107]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 719704480703361024, 'id_str'...",4769,False,This is Clyde. He's making sure you're having ...,NaN,...,NaN,NaN,NaN,NaN,1547,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2299,NaN,NaN,Tue Nov 17 22:06:57 +0000 2015,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666739322658398210, 'id_str'...",237,False,This is Lugan. He is a Bohemian Rhapsody. Very...,NaN,...,NaN,NaN,NaN,NaN,66,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
691,NaN,NaN,Wed Oct 12 02:53:11 +0000 2016,"[0, 130]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 779834320894713856, 'id_str'...",0,False,RT @dog_rates: This is Scout. He really wants ...,NaN,...,NaN,NaN,NaN,NaN,7766,False,{'created_at': 'Sun Sep 25 00:06:08 +0000 2016...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1595,NaN,NaN,Fri Jan 08 19:45:39 +0000 2016,"[0, 98]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 685547930707750912, 'id_str'...",34684,False,Everybody needs to read this. Jack is our firs...,NaN,...,NaN,NaN,NaN,NaN,16799,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1828,NaN,NaN,Sun Dec 13 02:51:51 +0000 2015,"[0, 144]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 675870715636224001, 'id_str'...",1698,False,&amp; this is Yoshi. Another world record cont...,NaN,...,NaN,NaN,NaN,NaN,601,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1152,NaN,NaN,Wed Apr 20 02:30:23 +0000 2016,"[0, 110]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 722613344872505345, 'id_str'...",5187,False,Meet Schnitzel. He's a Tropicana Floofboop. Ge...,NaN,...,NaN,NaN,NaN,NaN,1742,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1096,NaN,NaN,Sat May 21 00:50:46 +0000 2016,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 733822298390548480, 'id_str'...",3845,False,This is Moose. He's a Polynesian Floofer. Dapp...,NaN,...,NaN,NaN,NaN,NaN,1078,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
712,NaN,NaN,Mon Oct 03 15:42:44 +0000 2016,"[0, 97]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 782969083092471809, 'id_str'...",25811,False,This is Cali. She arrived preassembled. Conven...,NaN,...,NaN,NaN,NaN,NaN,8072,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835'

In [18]:
# View info of twitter_json dataframe
twitter_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 32 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2356 non-null object
display_text_range               2356 non-null object
entities                         2356 non-null object
extended_entities                2082 non-null object
favorite_count                   2356 non-null int64
favorited                        2356 non-null bool
full_text                        2356 non-null object
geo                              0 non-null float64
id                               2356 non-null int64
id_str                           2356 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null 

In [19]:
# View descriptive statistics of twitter_json dataframe
twitter_json.describe()

,contributors,coordinates,favorite_count,geo,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,quoted_status_id,quoted_status_id_str,retweet_count
count,0.0,0.0,2356.000000,0.0,2.356000e+03,2.356000e+03,7.700000e+01,7.700000e+01,7.700000e+01,7.700000e+01,2.600000e+01,2.600000e+01,2356.000000
mean,NaN,NaN,7949.753396,NaN,7.417684e+17,7.417684e+17,7.440692e+17,7.440692e+17,2.040329e+16,2.040329e+16,8.113972e+17,8.113972e+17,2927.556876
std,NaN,NaN,12325.762302,NaN,6.837209e+16,6.837209e+16,7.524295e+16,7.524295e+16,1.260797e+17,1.260797e+17,6.295843e+16,6.295843e+16,4936.906350
min,NaN,NaN,0.000000,NaN,6.660209e+17,6.660209e+17,6.658147e+17,6.658147e+17,1.185634e+07,1.185634e+07,6.721083e+17,6.721083e+17,0.000000
25%,NaN,NaN,1397.250000,NaN,6.776996e+17,6.776996e+17,6.757073e+17,6.757073e+17,3.589728e+08,3.589728e+08,7.761338e+17,7.761338e+17,582.000000
50%,NaN,NaN,3454.000000,NaN,7.178159e+17,7.178159e+17,7.032559e+17,7.032559e+17,4.196984e+09,4.196984e+09,8.281173e+17,8.281173e+17,1362.000000
75%,NaN,NaN,9757.500000,NaN,7.986755e+17,7.986755e+17,8.233264e+17,8.233264e+17,4.196984e+09,4.196984e+09,8.637581e+17,8.637581e+17,3414.250000
max,NaN,NaN,164611.000000,NaN,8.924206e+17,8.924206e+17,8.862664e+17,8.862664e+17,8.405479e+17,8.405479e+17,8.860534e+17,8.860534e+17,83874.000000


**Identify Duplicates for important data points**

In [20]:
# Investigate if duplicated Tweet IDs exist
twitter_archive.tweet_id.duplicated().sum()

0

In [21]:
twitter_images.tweet_id.duplicated().sum()

0

In [22]:
twitter_json.id.duplicated().sum()

14

In [23]:
# Identify duplicated IDs
twitter_json.id.value_counts()

666020888022790149    15
743510151680958465     1
825120256414846976     1
769212283578875904     1
700462010979500032     1
780858289093574656     1
699775878809702401     1
880095782870896641     1
760521673607086080     1
749075273010798592     1
776477788987613185     1
691820333922455552     1
715696743237730304     1
714606013974974464     1
760539183865880579     1
813157409116065792     1
676430933382295552     1
798644042770751489     1
833722901757046785     1
741099773336379392     1
818259473185828864     1
670704688707301377     1
667160273090932737     1
674394782723014656     1
672082170312290304     1
670093938074779648     1
759923798737051648     1
809920764300447744     1
805487436403003392     1
838085839343206401     1
                      ..
870308999962521604     1
720775346191278080     1
879492040517615616     1
775733305207554048     1
667911425562669056     1
834209720923721728     1
825026590719483904     1
758405701903519748     1
668986018524233728     1


In [24]:
# Investigate if duplicated jpg_url exist
twitter_images.jpg_url.duplicated().sum()

66

In [25]:
# Identify duplicated urls
twitter_images.jpg_url.value_counts()

https://pbs.twimg.com/media/Cbs3DOAXIAAp3Bd.jpg                                            2
https://pbs.twimg.com/media/Cs_DYr1XEAA54Pu.jpg                                            2
https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg                                            2
https://pbs.twimg.com/media/CiibOMzUYAA9Mxz.jpg                                            2
https://pbs.twimg.com/media/C2kzTGxWEAEOpPL.jpg                                            2
https://pbs.twimg.com/media/CvJCabcWgAIoUxW.jpg                                            2
https://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg                                            2
https://pbs.twimg.com/media/CiyHLocU4AI2pJu.jpg                                            2
https://pbs.twimg.com/tweet_video_thumb/CeBym7oXEAEWbEg.jpg                                2
https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg                                            2
https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg                       

In [26]:
twitter_images[twitter_images['jpg_url'] == 'https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg']

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1785,829374341691346946,https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg,1,Staffordshire_bullterrier,0.757547,True,American_Staffordshire_terrier,0.14995,True,Chesapeake_Bay_retriever,0.047523,True
1903,851953902622658560,https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg,1,Staffordshire_bullterrier,0.757547,True,American_Staffordshire_terrier,0.14995,True,Chesapeake_Bay_retriever,0.047523,True


In [27]:
twitter_images[twitter_images['jpg_url'] == 'https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg']

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
224,670319130621435904,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True


#### Step 2: Document

**a. Quality**

**Twitter Archive**
 1. dataset contains retweets
 2. in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be intergs instead of float (check if fields are essential for the analysis)
 3. timestamp and retweeted_status_timestamp should be datetime instead of object (string)
 4. incorrect dog names (for instance: 'such', 'a', 'an')
 5. the rating numerator and denominator have ratings above the standard scale (for instance numerator equal to 1776 and denominator equal to 170)
 6. urls appear in the source column
 7. source and dog stage datatype can be set to 'category'


**Twitter Images**
 1. Missing records: 2075 rows instead of 2356
 2. 66 tweet_ids have the same duplicated jpg_urls
 3. Natural network didn't recognize a dog in any of the attempts

**Twitter API**
 1. There are a few data type issues but for this project we need only 3 columns: Tweet ID, retweet count and favorite count
 2. Tweet ID 666020888022790149 is duplicated    

**b. Tidiness**

**General**
- All tables should be part of one dataset

**Twitter Archive**
- Different stage of dogs in columns instead of rows

**Twitter API**
- For this project we need only 3 columns: Tweet ID, retweet count and favorite count. Drop the rest of columns.

<a id='clean'></a>
### Part III - Cleaning Data

The very first thing to do before any cleaning occurs is to make a copy of each piece of data. All of the cleaning operations will be conducted on this copy so you can still view the original dirty and/or messy dataset later.

In [28]:
# Create copies of original DataFrames
twitter_archive_clean = twitter_archive.copy()
twitter_images_clean = twitter_images.copy()
twitter_json_clean = twitter_json.copy()

In [29]:
# We only want original ratings (no retweets) that have images so
# remove retweets from twitter_archive
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isna()]

In [30]:
# Drop columns related to retweets
twitter_archive_clean.drop(columns=['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' ], inplace=True)

**About 1. in_reply_to_status_id and 2. in_reply_to_user_id**
1. If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s ID
2. If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet. 

In [31]:
twitter_archive_clean.in_reply_to_status_id.value_counts()

6.671522e+17    2
8.562860e+17    1
8.131273e+17    1
6.754971e+17    1
6.827884e+17    1
8.265984e+17    1
6.780211e+17    1
6.689207e+17    1
6.658147e+17    1
6.737159e+17    1
7.590995e+17    1
8.862664e+17    1
7.384119e+17    1
7.727430e+17    1
7.468859e+17    1
8.634256e+17    1
6.693544e+17    1
6.914169e+17    1
6.920419e+17    1
6.753494e+17    1
7.291135e+17    1
8.406983e+17    1
6.747400e+17    1
7.501805e+17    1
6.744689e+17    1
7.638652e+17    1
6.747934e+17    1
8.503288e+17    1
6.747522e+17    1
8.816070e+17    1
               ..
8.380855e+17    1
8.211526e+17    1
8.558616e+17    1
8.558585e+17    1
7.032559e+17    1
6.678065e+17    1
8.018543e+17    1
7.667118e+17    1
6.855479e+17    1
6.717299e+17    1
6.715610e+17    1
6.758457e+17    1
6.924173e+17    1
7.476487e+17    1
8.381455e+17    1
6.903413e+17    1
8.476062e+17    1
8.352460e+17    1
6.813394e+17    1
8.795538e+17    1
6.860340e+17    1
8.571567e+17    1
6.765883e+17    1
7.044857e+17    1
8.707262e+

In [32]:
twitter_archive_clean.in_reply_to_user_id.value_counts()

4.196984e+09    47
2.195506e+07     2
7.305050e+17     1
2.916630e+07     1
3.105441e+09     1
2.918590e+08     1
2.792810e+08     1
2.319108e+09     1
1.806710e+08     1
3.058208e+07     1
2.625958e+07     1
1.943518e+08     1
3.589728e+08     1
8.405479e+17     1
2.894131e+09     1
2.143566e+07     1
2.281182e+09     1
1.648776e+07     1
4.717297e+09     1
2.878549e+07     1
1.582854e+09     1
4.670367e+08     1
4.738443e+07     1
1.361572e+07     1
1.584641e+07     1
2.068372e+07     1
1.637468e+07     1
1.185634e+07     1
1.198989e+09     1
1.132119e+08     1
7.759620e+07     1
Name: in_reply_to_user_id, dtype: int64

**Conclusions**
- 47 out of 77 tweets were replies to the original post of the tweet's author id: 4196983835, meaning tweets of @dog_rates
- Since I do not find these fields essential for the dataset, I am going to drop them

In [33]:
# Drop columns related to tweets that are reply to an original tweet (mainly coming from @dog_rates)
twitter_archive_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id'], inplace=True)

In [34]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(9)
memory usage: 220.9+ KB


In [35]:
# Change datatype of timestamp from object to timestamp
twitter_archive_clean.timestamp = pd.to_datetime(twitter_archive_clean.timestamp)

In [36]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 220.9+ KB


In [37]:
# Print column names for melt
columns = twitter_archive_clean.columns.tolist()
print(columns[:-4])

['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name']


In [38]:
# Check dog stage values before melting
print(twitter_archive_clean.doggo.value_counts())
print(twitter_archive_clean.floofer.value_counts())
print(twitter_archive_clean.pupper.value_counts())
print(twitter_archive_clean.puppo.value_counts())

None     2088
doggo      87
Name: doggo, dtype: int64
None       2165
floofer      10
Name: floofer, dtype: int64
None      1941
pupper     234
Name: pupper, dtype: int64
None     2150
puppo      25
Name: puppo, dtype: int64


In [39]:
# Melt dog stages into one column
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars=['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name'],
                           var_name='stage', value_name='dog_stage')

In [40]:
# Drop duplicates after pivoting
twitter_archive_clean = twitter_archive_clean.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

In [41]:
# Check if we did not miss any status
twitter_archive_clean.dog_stage.value_counts()

None       1831
pupper      234
doggo        75
puppo        25
floofer      10
Name: dog_stage, dtype: int64

In [42]:
# Drop stage column
twitter_archive_clean.drop(columns=['stage'], inplace=True)

In [43]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 2095 to 7298
Data columns (total 9 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
dog_stage             2175 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 169.9+ KB


In [44]:
# Enhance missing values in the dog stage by extracting statuses from text
twitter_archive_clean['dog_stage2'] = twitter_archive_clean['text'].str.extract('(puppo|pupper|floofer|doggo)', expand=True)

In [45]:
# Check results of the extract and compare with the results of the melt
twitter_archive_clean['dog_stage2'].value_counts()

pupper     244
doggo       84
puppo       32
floofer      4
Name: dog_stage2, dtype: int64

In [46]:
# Check 50 random results
twitter_archive_clean.sample(50)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,dog_stage2
15,889278841981685760,2017-07-24 00:19:32,"<a href=""http://twitter.com/download/iphone"" r...",This is Oliver. You're witnessing one of his m...,https://twitter.com/dog_rates/status/889278841...,13,10,Oliver,None,NaN
753,755206590534418437,2016-07-19 01:04:16,"<a href=""http://twitter.com/download/iphone"" r...",This is one of the most inspirational stories ...,https://twitter.com/dog_rates/status/755206590...,14,10,one,doggo,doggo
2118,667065535570550784,2015-11-18 19:43:11,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Hufflepuff. Loves vest. Eyes wi...,https://twitter.com/dog_rates/status/667065535...,8,10,None,None,NaN
4203,668627278264475648,2015-11-23 03:09:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Timofy. He's a pilot for Southwest. It...,https://twitter.com/dog_rates/status/668627278...,9,10,Timofy,None,NaN
3219,714258258790387713,2016-03-28 01:10:13,"<a href=""http://twitter.com/download/iphone"" r...",Meet Travis and Flurp. Travis is pretty chill ...,https://twitter.com/dog_rates/status/714258258...,10,10,Travis,None,NaN
3224,713909862279876608,2016-03-27 02:05:49,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Doug. His nose is legendary af. 12/10 ...,https://vine.co/v/iDWlapaXWmm,12,10,Doug,None,NaN
2513,824663926340194305,2017-01-26 17:02:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Wilson. Named after the volleyball. He...,https://twitter.com/dog_rates/status/824663926...,13,10,Wilson,None,NaN
3031,745074613265149952,2016-06-21 02:03:25,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Jeffrey. He wasn't prepared to execute...,https://vine.co/v/iQm3JAXuFmv,11,10,Jeffrey,None,NaN
2727,788039637453406209,2016-10-17 15:31:05,"<a href=""http://twitter.com/download/iphone"" r...",Did... did they pick out that license plate? 1...,https://twitter.com/dog_rates/status/788039637...,12,10,None,None,NaN
2155,666273097616637952,2015-11-16 15:14:19,"<a href=""http://twitter.com/download/iphone"" r...",Can take selfies 11/10 https://t.co/ws2AMaNwPW,https://twitter.com/dog_rates/status/666273097...,11,10,None,None,NaN


In [47]:
# Since there are instances where the extract brought more statuses, we are going to "merge" values from two columns
# if 'dog stage' is equal to 'None', take value from the 'dog stage2'. If not equal to 'None', keep the original value
twitter_archive_clean['dog_stage3'] = np.where(twitter_archive_clean['dog_stage'] == 'None', twitter_archive_clean['dog_stage2'], twitter_archive_clean['dog_stage'])

In [48]:
twitter_archive_clean['dog_stage3'].value_counts()

pupper     258
doggo       83
puppo       32
floofer     10
Name: dog_stage3, dtype: int64

In [49]:
# Drop unneccessary columns
twitter_archive_clean.drop(columns=['dog_stage', 'dog_stage2'], inplace=True)

In [50]:
# Rename column back to 'dog_stage'
twitter_archive_clean.rename(columns={'dog_stage3':'dog_stage'}, inplace=True)

In [51]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 2095 to 7298
Data columns (total 9 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
dog_stage             383 non-null object
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 169.9+ KB


In [52]:
# Check dog names started with the lowercase
lowercase = twitter_archive_clean['name'].where(twitter_archive_clean['name'].str.islower())
lowercase.value_counts()

a               55
the              8
an               6
very             4
one              4
quite            3
just             3
getting          2
actually         2
not              2
by               1
space            1
old              1
my               1
his              1
mad              1
infuriating      1
such             1
officially       1
life             1
all              1
incredibly       1
unacceptable     1
this             1
light            1
Name: name, dtype: int64

In [53]:
# Replace all instances of lowercase words that are not dog names with 'None'
twitter_archive_clean['name'].replace('a', 'None', inplace=True)
twitter_archive_clean['name'].replace('the', 'None', inplace=True)
twitter_archive_clean['name'].replace('an', 'None', inplace=True)
twitter_archive_clean['name'].replace('one', 'None', inplace=True)
twitter_archive_clean['name'].replace('very', 'None', inplace=True)
twitter_archive_clean['name'].replace('just', 'None', inplace=True)
twitter_archive_clean['name'].replace('quite', 'None', inplace=True)
twitter_archive_clean['name'].replace('not', 'None', inplace=True)
twitter_archive_clean['name'].replace('getting', 'None', inplace=True)
twitter_archive_clean['name'].replace('actually', 'None', inplace=True)
twitter_archive_clean['name'].replace('light', 'None', inplace=True)
twitter_archive_clean['name'].replace('old', 'None', inplace=True)
twitter_archive_clean['name'].replace('incredibly', 'None', inplace=True)
twitter_archive_clean['name'].replace('my', 'None', inplace=True)
twitter_archive_clean['name'].replace('such', 'None', inplace=True)
twitter_archive_clean['name'].replace('officially', 'None', inplace=True)
twitter_archive_clean['name'].replace('unacceptable', 'None', inplace=True)
twitter_archive_clean['name'].replace('life', 'None', inplace=True)
twitter_archive_clean['name'].replace('by', 'None', inplace=True)
twitter_archive_clean['name'].replace('his', 'None', inplace=True)
twitter_archive_clean['name'].replace('infuriating', 'None', inplace=True)
twitter_archive_clean['name'].replace('mad', 'None', inplace=True)
twitter_archive_clean['name'].replace('space', 'None', inplace=True)
twitter_archive_clean['name'].replace('this', 'None', inplace=True)
twitter_archive_clean['name'].replace('all', 'None', inplace=True)

In [54]:
# Check
twitter_archive_clean['name'].value_counts()

None           784
Charlie         11
Lucy            11
Cooper          10
Oliver          10
Tucker           9
Penny            9
Winston          8
Sadie            8
Lola             8
Toby             7
Daisy            7
Bella            6
Bailey           6
Koda             6
Bo               6
Stanley          6
Jax              6
Oscar            6
Buddy            5
Bentley          5
Milo             5
Leo              5
Chester          5
Rusty            5
Scout            5
Louis            5
Dave             5
Larry            4
Archie           4
              ... 
Snicku           1
Fizz             1
Ralphie          1
Pip              1
Jeffrie          1
Keet             1
Edmund           1
Duchess          1
Hamrick          1
Richie           1
Tuck             1
Kellogg          1
Arlen            1
Furzey           1
Timofy           1
Jeremy           1
Zoe              1
Tuco             1
Lolo             1
Blipson          1
Willie           1
Sid         

In [55]:
# increase the width of colums so the text is fully displayed
pd.set_option('display.max_colwidth', -1)

In [56]:
# check if rating numerators above 15 are valid
print(twitter_archive_clean[twitter_archive_clean.rating_numerator > 15]['text'])

52      @roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s                                                                              
3429    Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ                                                                             
3631    Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55                             
3630    Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3                                           
3659    I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible                              
3250    Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7

In [57]:
# check if rating denominators above 15 are valid
print(twitter_archive_clean[twitter_archive_clean.rating_denominator > 10]['text'])

3429    Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ                                                      
3631    Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55      
3630    Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3                    
3659    I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible       
3658    This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5       
3594    Yes I do realize a rating of 4/20 would've been fitting. However, it would be unjust to give these cooperative pups that low of a rating          
3250    Here's a brigade of puppers. All look very prepared for whatev

#### Conclusion
1. There are ratings where numerator and denominator are above "the standard" scale
2. There are cases where rating numbers are floats

In [58]:
# change data type for numerator and denominator to float
twitter_archive_clean['rating_numerator'] = twitter_archive_clean['rating_numerator'].astype(float)
twitter_archive_clean['rating_denominator'] = twitter_archive_clean['rating_denominator'].astype(float)

In [59]:
# check datatypes
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 2095 to 7298
Data columns (total 9 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null float64
rating_denominator    2175 non-null float64
name                  2175 non-null object
dog_stage             383 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 169.9+ KB


In [60]:
# check tweets with decimals in rating
twitter_archive_clean[twitter_archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
42,883482846933004288,2017-07-08 00:28:19,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948","https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",5.0,10.0,Bella,NaN
3685,681340665377193984,2015-12-28 05:07:27,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,NaN,5.0,10.0,None,NaN
3708,680494726643068929,2015-12-25 21:06:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,https://twitter.com/dog_rates/status/680494726643068929/photo/1,26.0,10.0,None,pupper
2733,786709082849828864,2016-10-13 23:23:56,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",https://twitter.com/dog_rates/status/786709082849828864/photo/1,75.0,10.0,Logan,NaN
4967,778027034220126208,2016-09-20 00:24:34,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,https://twitter.com/dog_rates/status/778027034220126208/photo/1,27.0,10.0,Sophie,pupper


In [61]:
# change rating numerator to correct values from the text
twitter_archive_clean.loc[(twitter_archive_clean['tweet_id'] == 883482846933004288) & (twitter_archive_clean['rating_numerator'] == 5), ['rating_numerator']] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean['tweet_id'] == 681340665377193984) & (twitter_archive_clean['rating_numerator'] == 5), ['rating_numerator']] = 9.5
twitter_archive_clean.loc[(twitter_archive_clean['tweet_id'] == 680494726643068929) & (twitter_archive_clean['rating_numerator'] == 26), ['rating_numerator']] = 11.26
twitter_archive_clean.loc[(twitter_archive_clean['tweet_id'] == 786709082849828864) & (twitter_archive_clean['rating_numerator'] == 75), ['rating_numerator']] = 9.75
twitter_archive_clean.loc[(twitter_archive_clean['tweet_id'] == 778027034220126208) & (twitter_archive_clean['rating_numerator'] == 27), ['rating_numerator']] = 11.27

In [62]:
# check if values were correctly replaced
twitter_archive_clean[twitter_archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
42,883482846933004288,2017-07-08 00:28:19,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948","https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",13.50,10.0,Bella,NaN
3685,681340665377193984,2015-12-28 05:07:27,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,NaN,9.50,10.0,None,NaN
3708,680494726643068929,2015-12-25 21:06:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,https://twitter.com/dog_rates/status/680494726643068929/photo/1,11.26,10.0,None,pupper
2733,786709082849828864,2016-10-13 23:23:56,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",https://twitter.com/dog_rates/status/786709082849828864/photo/1,9.75,10.0,Logan,NaN
4967,778027034220126208,2016-09-20 00:24:34,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,https://twitter.com/dog_rates/status/778027034220126208/photo/1,11.27,10.0,Sophie,pupper


In [63]:
# check the occurences of source type
twitter_archive_clean['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2042
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                        91  
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     31  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    11  
Name: source, dtype: int64

In [64]:
# Remove all instances of url from source
twitter_archive_clean['source'].replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Twitter for iPhone', inplace=True)
twitter_archive_clean['source'].replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>', 'Vine - Make a Scene', inplace=True)
twitter_archive_clean['source'].replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Twitter Web Client', inplace=True)
twitter_archive_clean['source'].replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'TweetDeck', inplace=True)

In [65]:
# check if values were correctly replaced
twitter_archive_clean['source'].value_counts()

Twitter for iPhone     2042
Vine - Make a Scene    91  
Twitter Web Client     31  
TweetDeck              11  
Name: source, dtype: int64

In [66]:
# Change source and dog stage datatype to category
twitter_archive_clean['source'] = twitter_archive_clean['source'].astype('category')
twitter_archive_clean['dog_stage'] = twitter_archive_clean['dog_stage'].astype('category')

In [67]:
# check twitter_json_clean data - columns to be removed
twitter_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 32 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2356 non-null object
display_text_range               2356 non-null object
entities                         2356 non-null object
extended_entities                2082 non-null object
favorite_count                   2356 non-null int64
favorited                        2356 non-null bool
full_text                        2356 non-null object
geo                              0 non-null float64
id                               2356 non-null int64
id_str                           2356 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null 

In [68]:
# drop unneccessary columns
twitter_json_clean.drop(['contributors',
'coordinates',
'created_at',
'display_text_range',
'entities',
'extended_entities',
'favorited',
'full_text',
'geo',
'id_str',
'in_reply_to_screen_name',
'in_reply_to_status_id',
'in_reply_to_status_id_str',
'in_reply_to_user_id',
'in_reply_to_user_id_str',
'is_quote_status',
'lang',
'place',
'possibly_sensitive',
'possibly_sensitive_appealable',
'quoted_status',
'quoted_status_id',
'quoted_status_id_str',
'quoted_status_permalink',
'retweeted',
'retweeted_status',
'source',
'truncated',
'user'], axis=1,inplace=True)

In [69]:
# recheck the dataset
twitter_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
favorite_count    2356 non-null int64
id                2356 non-null int64
retweet_count     2356 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


In [70]:
twitter_json_clean['id'].value_counts()

666020888022790149    15
743510151680958465    1 
825120256414846976    1 
769212283578875904    1 
700462010979500032    1 
780858289093574656    1 
699775878809702401    1 
880095782870896641    1 
760521673607086080    1 
749075273010798592    1 
776477788987613185    1 
691820333922455552    1 
715696743237730304    1 
714606013974974464    1 
760539183865880579    1 
813157409116065792    1 
676430933382295552    1 
798644042770751489    1 
833722901757046785    1 
741099773336379392    1 
818259473185828864    1 
670704688707301377    1 
667160273090932737    1 
674394782723014656    1 
672082170312290304    1 
670093938074779648    1 
759923798737051648    1 
809920764300447744    1 
805487436403003392    1 
838085839343206401    1 
                     .. 
870308999962521604    1 
720775346191278080    1 
879492040517615616    1 
775733305207554048    1 
667911425562669056    1 
834209720923721728    1 
825026590719483904    1 
758405701903519748    1 
668986018524233728    1 


In [71]:
# Drop duplicated twitter id
twitter_json_clean = twitter_json_clean.sort_values('id').drop_duplicates('id', keep = 'last')

In [72]:
# recheck if duplicates still exist in the ID column
twitter_json_clean.id.duplicated().sum()

0

In [73]:
# rename id column to 'tweet_id'
twitter_json_clean.rename(columns={'id':'tweet_id'}, inplace=True)

In [74]:
twitter_images_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [75]:
# identify records where the prediction didn't recognize a dog at the 1st stage 
# then use the 2nd or 3rd prediction if the dog has been found

dog = []
conf = []

def dogs_prediction(row):
    if row['p1_dog'] == True:
        dog.append(row['p1'])
        conf.append(row['p1_conf'])
    elif row['p2_dog'] == True:
        dog.append(row['p2'])
        conf.append(row['p2_conf'])
    elif row['p3_dog'] == True:
        dog.append(row['p3'])
        conf.append(row['p3_conf'])
    else:
        dog.append(np.nan)
        conf.append(np.nan)

In [76]:
# apply and check the lenght of the results
twitter_images_clean.apply(dogs_prediction, axis=1)
len(dog), len(conf), len(twitter_images_clean)

(2075, 2075, 2075)

In [77]:
# add new columns to the dataset
twitter_images_clean['dog'] = dog
twitter_images_clean['conf'] = conf

In [78]:
# Remove underscore from and capitalize dog types 
twitter_images_clean['dog'] = twitter_images_clean['dog'].str.replace("_", ' ')
twitter_images_clean['dog'] = twitter_images_clean['dog'].str.title()

In [79]:
twitter_images_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,dog,conf
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,Welsh Springer Spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True,Redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,German Shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,Rhodesian Ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,Miniature Pinscher,0.560311


In [80]:
# drop unneccessary columns from twitter_images_clean
twitter_images_clean.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 
                           'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis=1,inplace=True)

In [81]:
twitter_images_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 4 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
dog         1751 non-null object
conf        1751 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 64.9+ KB


In [82]:
#joing datasources

df_merged = pd.merge(twitter_archive_clean, twitter_json_clean, on = ['tweet_id'], how = 'inner')
master_df = pd.merge(df_merged, twitter_images_clean, on = ['tweet_id'], how = 'inner')

In [83]:
master_df.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,dog,conf
0,667443425659232256,2015-11-19 20:44:47,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsessed with bread. Waddles. Flies sometimes (wow!). Very happy dog. 6/10 https://t.co/rqO4I3nf2N,https://twitter.com/dog_rates/status/667443425659232256/photo/1,6.0,10.0,None,NaN,789,590,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,NaN,NaN
1,667453023279554560,2015-11-19 21:22:56,Twitter Web Client,Meet Cupcake. I would do unspeakable things for Cupcake. 11/10 https://t.co/6uLCWR9Efa,https://twitter.com/dog_rates/status/667453023279554560/photo/1,11.0,10.0,Cupcake,NaN,315,91,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,Labrador Retriever,0.825670
2,667455448082227200,2015-11-19 21:32:34,Twitter Web Client,This is Reese and Twips. Reese protects Twips. Both think they're too good for seat belts. Simply reckless. 7/10s https://t.co/uLzRi1drVK,https://twitter.com/dog_rates/status/667455448082227200/photo/1,7.0,10.0,Reese,NaN,194,60,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,Tibetan Terrier,0.676376
3,667470559035432960,2015-11-19 22:32:36,Twitter Web Client,This is a northern Wahoo named Kohl. He runs this town. Chases tumbleweeds. Draws gun wicked fast. 11/10 legendary https://t.co/J4vn2rOYFk,https://twitter.com/dog_rates/status/667470559035432960/photo/1,11.0,10.0,None,NaN,259,100,https://pbs.twimg.com/media/CUNU78YWEAECmpB.jpg,Toy Poodle,0.304175
4,667491009379606528,2015-11-19 23:53:52,Twitter Web Client,Two dogs in this one. Both are rare Jujitsu Pythagoreans. One slightly whiter than other. Long legs. 7/10 and 8/10 https://t.co/ITxxcc4v9y,https://twitter.com/dog_rates/status/667491009379606528/photo/1,7.0,10.0,None,NaN,534,232,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,Borzoi,0.852088


In [84]:
# set the tweet id to object type
master_df['tweet_id'] = master_df['tweet_id'].astype(object)

In [85]:
# replace 'None' dog names with NaN
master_df['name'].replace('None', np.nan, inplace=True)

In [86]:
# give columns more meaningful names
master_df = master_df.rename(columns = {'timestamp': 'tweet_datetime', 'source': 'tweet_source', 'text': 'tweet_content', 
                                        'expanded_urls': 'tweet_url', 'name': 'dog_name', 'jpg_url': 'prediction_picture',
                                        'dog': 'prediction_breed', 'conf': 'prediction_confidence'})

In [87]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 1992
Data columns (total 14 columns):
tweet_id                 1993 non-null object
tweet_datetime           1993 non-null datetime64[ns]
tweet_source             1993 non-null category
tweet_content            1993 non-null object
tweet_url                1993 non-null object
rating_numerator         1993 non-null float64
rating_denominator       1993 non-null float64
dog_name                 1349 non-null object
dog_stage                340 non-null category
favorite_count           1993 non-null int64
retweet_count            1993 non-null int64
prediction_picture       1993 non-null object
prediction_breed         1685 non-null object
prediction_confidence    1685 non-null float64
dtypes: category(2), datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 206.7+ KB


In [88]:
# write to csv
master_df.to_csv('twitter_archive_master.csv', index=False)